## WORK IN PROGRESS


In [1]:
import os
import numpy as np
import kagglehub
import pandas as pd
import os
import ta
import shutil
import os

from torch import nn
from stable_baselines3 import SAC
from stable_baselines3.common.callbacks import (
    EvalCallback,
    CheckpointCallback,
)
from stable_baselines3.common.buffers import *
from stable_baselines3.common.vec_env import DummyVecEnv, VecMonitor
from stable_baselines3 import PPO, A2C, DDPG, SAC, TD3
import gymnasium as gym
import gym_anytrading
import matplotlib.pyplot as plt
import heapq
import random
import torch
from scipy.spatial import KDTree
#chage point stuff
import ruptures as rpt
from sklearn.cluster import KMeans

In [2]:
# SER using change point detection
# we treat changing/different market regimes as sequential tasks
# labels that have the same tag as the current regime will have higher priority in long-term memory; when there is a change point the scoring changes entirely

# considering combination of matching global distribution w/ random normal dist and matching regimes detected by center-point detection

class SERReplayBuffer(BaseBuffer):
    def __init__(
        self,
        buffer_size: int,
        observation_space: spaces.Space,
        action_space: spaces.Space,
        priority_queue_size: int,
        priority_queue_percent: float, # i.e. 10% use --> 0.1
        device: Union[th.device, str] = "auto",
        n_envs: int = 1,
        optimize_memory_usage: bool = False,
        handle_timeout_termination: bool = True,
    ):
        super().__init__(buffer_size, observation_space, action_space, device, n_envs=n_envs)

        # Adjust buffer size
        self.buffer_size = max(buffer_size // n_envs, 1)

        # Check that the replay buffer can fit into the memory
        if psutil is not None:
            mem_available = psutil.virtual_memory().available

        # there is a bug if both optimize_memory_usage and handle_timeout_termination are true
        # see https://github.com/DLR-RM/stable-baselines3/issues/934
        if optimize_memory_usage and handle_timeout_termination:
            raise ValueError(
                "ReplayBuffer does not support optimize_memory_usage = True "
                "and handle_timeout_termination = True simultaneously."
            )
        self.optimize_memory_usage = optimize_memory_usage

        self.observations = np.zeros((self.buffer_size, self.n_envs, *self.obs_shape), dtype=observation_space.dtype)

        if not optimize_memory_usage:
            # When optimizing memory, `observations` contains also the next observation
            self.next_observations = np.zeros((self.buffer_size, self.n_envs, *self.obs_shape), dtype=observation_space.dtype)

        self.actions = np.zeros(
            (self.buffer_size, self.n_envs, self.action_dim), dtype=self._maybe_cast_dtype(action_space.dtype)
        )

        self.rewards = np.zeros((self.buffer_size, self.n_envs), dtype=np.float32)
        self.dones = np.zeros((self.buffer_size, self.n_envs), dtype=np.float32)
        # Handle timeouts termination properly if needed
        # see https://github.com/DLR-RM/stable-baselines3/issues/284
        self.handle_timeout_termination = handle_timeout_termination
        self.timeouts = np.zeros((self.buffer_size, self.n_envs), dtype=np.float32)

        if psutil is not None:
            total_memory_usage: float = (
                self.observations.nbytes + self.actions.nbytes + self.rewards.nbytes + self.dones.nbytes
            )

            if not optimize_memory_usage:
                total_memory_usage += self.next_observations.nbytes

            if total_memory_usage > mem_available:
                # Convert to GB
                total_memory_usage /= 1e9
                mem_available /= 1e9
                warnings.warn(
                    "This system does not have apparently enough memory to store the complete "
                    f"replay buffer {total_memory_usage:.2f}GB > {mem_available:.2f}GB"
                )
        
        self.priority_queue_size= priority_queue_size
        self.long_term_memory =[]  # (score, (obs, next_obs, action, reward, done))
        self.priority_queue_percent = priority_queue_percent
        self.curr_trend = None        

    def add(
        self,
        obs: np.ndarray,
        next_obs: np.ndarray,
        action: np.ndarray,
        reward: np.ndarray,
        done: np.ndarray,
        infos: list[dict[str, Any]],
    ) -> None:
        # Reshape needed when using multiple envs with discrete observations
        # as numpy cannot broadcast (n_discrete,) to (n_discrete, 1)
        if isinstance(self.observation_space, spaces.Discrete):
            obs = obs.reshape((self.n_envs, *self.obs_shape))
            next_obs = next_obs.reshape((self.n_envs, *self.obs_shape))

        # Reshape to handle multi-dim and discrete action spaces, see GH #970 #1392
        action = action.reshape((self.n_envs, self.action_dim))

        # Copy to avoid modification by reference
        self.observations[self.pos] = np.array(obs)

        if self.optimize_memory_usage:
            self.observations[(self.pos + 1) % self.buffer_size] = np.array(next_obs)
        else:
            self.next_observations[self.pos] = np.array(next_obs)

        self.actions[self.pos] = np.array(action)
        self.rewards[self.pos] = np.array(reward)
        self.dones[self.pos] = np.array(done)

        if self.handle_timeout_termination:
            self.timeouts[self.pos] = np.array([info.get("TimeLimit.truncated", False) for info in infos])

        self.pos += 1
        if self.pos == self.buffer_size:
            self.full = True
            self.pos = 0

        # compute score using given strategy and push to priority queue
        # will get rid of the lowest score among the ones stored in long term mem if we exceed memory limit
        
        trend_ids = [info.get("trend_id", None) for info in infos]
        for trend_id in trend_ids:
            if trend_id is not None and (self.curr_trend is None or trend_id != self.curr_trend):
                print("set_curr_trend")
                self.set_curr_trend(trend_id)
                #break

        idx = (self.pos - 1) % self.buffer_size
        for env_idx in range(self.n_envs):
            score = float(np.random.normal())  
            heapq.heappush(self.long_term_memory, (score, idx, env_idx, self.curr_trend))
        if len(self.long_term_memory) > self.priority_queue_size:
            heapq.heappop(self.long_term_memory)

    def sample(self, batch_size: int, env: Optional[VecNormalize] = None):
        #print("sampling")
        long_size=int(batch_size*self.priority_queue_percent)
        fifo_size=batch_size-long_size
        
        # sample from FIFO
        if not self.optimize_memory_usage:
            return super().sample(batch_size=batch_size, env=env)
        # Do not sample the element with index `self.pos` as the transitions is invalid
        # (we use only one array to store `obs` and `next_obs`)
        if self.full:
            fifo_inds = (np.random.randint(1, self.buffer_size, size=fifo_size) + self.pos) % self.buffer_size
        else:
            fifo_inds = np.random.randint(0, self.pos, size=fifo_size)

        fifo_samples = self._get_samples(fifo_inds, env=env)

        # sample from long term memoru
        if len(self.long_term_memory) >= long_size:
            sampled_long_mem = random.sample(self.long_term_memory, long_size)
        else:
            sampled_long_mem = self.long_term_memory

        buffer_idxs = [idx for _, idx, env in sampled_long_mem]
        env_idxs = [env for _, idx, env in sampled_long_mem]

        long_obs = self.observations[buffer_idxs, env_idxs]
        if self.optimize_memory_usage:
            long_next_obs = self.observations[(np.array(buffer_idxs) + 1) % self.buffer_size, env_idxs]
        else:
            long_next_obs = self.next_observations[buffer_idxs, env_idxs]

        long_actions = self.actions[buffer_idxs, env_idxs]
        long_rewards = self.rewards[buffer_idxs, env_idxs].reshape(-1, 1)
        long_dones = (self.dones[buffer_idxs, env_idxs] * (1 - self.timeouts[buffer_idxs, env_idxs])).reshape(-1, 1)

        long_obs = self.to_torch(self._normalize_obs(long_obs, env))
        long_next_obs = self.to_torch(self._normalize_obs(long_next_obs, env))
        long_actions = self.to_torch(long_actions)
        long_rewards = self.to_torch(self._normalize_reward(long_rewards, env))
        long_dones = self.to_torch(long_dones)

        # combine FIFO + long-term mem
        obs = torch.cat([fifo_samples.observations, long_obs], dim=0)
        next_obs = torch.cat([fifo_samples.next_observations, long_next_obs], dim=0)
        actions = torch.cat([fifo_samples.actions, long_actions], dim=0)
        rewards = torch.cat([fifo_samples.rewards, long_rewards], dim=0)
        dones = torch.cat([fifo_samples.dones, long_dones], dim=0)

        return ReplayBufferSamples(obs, actions, next_obs, dones, rewards)

    def _get_samples(self, batch_inds: np.ndarray, env: Optional[VecNormalize] = None) -> ReplayBufferSamples:
        # Sample randomly the env idx
        env_indices = np.random.randint(0, high=self.n_envs, size=(len(batch_inds),))

        if self.optimize_memory_usage:
            next_obs = self._normalize_obs(self.observations[(batch_inds + 1) % self.buffer_size, env_indices, :], env)
        else:
            next_obs = self._normalize_obs(self.next_observations[batch_inds, env_indices, :], env)

        data = (
            self._normalize_obs(self.observations[batch_inds, env_indices, :], env),
            self.actions[batch_inds, env_indices, :],
            next_obs,
            # Only use dones that are not due to timeouts
            # deactivated by default (timeouts is initialized as an array of False)
            (self.dones[batch_inds, env_indices] * (1 - self.timeouts[batch_inds, env_indices])).reshape(-1, 1),
            self._normalize_reward(self.rewards[batch_inds, env_indices].reshape(-1, 1), env),
        )
        #print("sampling done")
        return ReplayBufferSamples(*tuple(map(self.to_torch, data)))
    
    def set_curr_trend(self, trend_id):
        print("in set_curr_trend")
        if self.curr_trend != trend_id:
            self.curr_trend = trend_id
            print("start rescore")
            self.rescore_mem()

    def rescore_mem(self):
        # resocre entire long term memory if the current trend does not match the trend on the stored 
        #print("rescoring memory for {self.current_trend_id}")
        updated_mem = []
        for score, idx, env_idx, trend_idx in self.long_term_memory:
            new_score = score
            if trend_idx == self.curr_trend:
                new_score*=1.5
            else:
                new_score*=0.5

            updated_mem.append((new_score, idx, env_idx, trend_idx))
        print("heapify")
        heapq.heapify(updated_mem)
        print("heapify done")
        self.long_term_memory = updated_mem

    @staticmethod
    def _maybe_cast_dtype(dtype: np.typing.DTypeLike) -> np.typing.DTypeLike:
        """
        Cast `np.float64` action datatype to `np.float32`,
        keep the others dtype unchanged.
        See GH#1572 for more information.

        :param dtype: The original action space dtype
        :return: ``np.float32`` if the dtype was float64,
            the original dtype otherwise.
        """
        if dtype == np.float64:
            return np.float32
        return dtype

In [3]:
# Download latest version
# path = kagglehub.dataset_download("debashis74017/algo-trading-data-nifty-100-data-with-indicators")
path_sp500 = kagglehub.dataset_download(
    "gratefuldata/intraday-stock-data-1-min-sp-500-200821"
)


print("Path to dataset files:", path_sp500)

print("\nFiles in S&P 500 dataset directory:")
try:
    files_sp500 = os.listdir(path_sp500)
    print(files_sp500)
except FileNotFoundError:
    print(f"Error: Directory not found at {path_sp500}")
except Exception as e:
    print(f"An error occurred: {e}")

local_dir = "./datasets/sp500"
os.makedirs(local_dir, exist_ok=True)  # Create the directory if it doesn't exist

for fname in files_sp500:
    src = os.path.join(path_sp500, fname)
    dst = os.path.join(local_dir, fname)
    shutil.copy2(src, dst)  # Use copy2 to preserve metadata

print(f"Files saved to {local_dir}")

full_csv_path = os.path.join(path_sp500, files_sp500[0])
output_path = "sp500_with_ta_features.csv"


if os.path.exists(output_path):
    df = pd.read_csv(output_path)
    # Ensure datetime column is correctly parsed when loading from CSV
    df["datetime"] = pd.to_datetime(df["datetime"])
else:
    df = pd.read_csv(full_csv_path) 
    df["datetime"] = pd.to_datetime(df["date"])

    start_date = pd.Timestamp("2018-01-01")
    end_date = pd.Timestamp("2021-12-31 23:59:59")
    df = df[(df["datetime"] >= start_date) & (df["datetime"] <= end_date)]
    df = df.sort_values("datetime")

    df = ta.add_all_ta_features(
        df,
        open="open",
        high="high",
        low="low",
        close="close",
        volume="volume",
        fillna=True,
    )

    df.to_csv(output_path, index=False)

print("DataFrame columns:", df.columns.tolist())

Path to dataset files: /Users/melisacivelekoglu/.cache/kagglehub/datasets/gratefuldata/intraday-stock-data-1-min-sp-500-200821/versions/1

Files in S&P 500 dataset directory:
['1_min_SPY_2008-2021.csv']
Files saved to ./datasets/sp500
DataFrame columns: ['Unnamed: 0', 'date', 'open', 'high', 'low', 'close', 'volume', 'barCount', 'average', 'datetime', 'volume_adi', 'volume_obv', 'volume_cmf', 'volume_fi', 'volume_em', 'volume_sma_em', 'volume_vpt', 'volume_vwap', 'volume_mfi', 'volume_nvi', 'volatility_bbm', 'volatility_bbh', 'volatility_bbl', 'volatility_bbw', 'volatility_bbp', 'volatility_bbhi', 'volatility_bbli', 'volatility_kcc', 'volatility_kch', 'volatility_kcl', 'volatility_kcw', 'volatility_kcp', 'volatility_kchi', 'volatility_kcli', 'volatility_dcl', 'volatility_dch', 'volatility_dcm', 'volatility_dcw', 'volatility_dcp', 'volatility_atr', 'volatility_ui', 'trend_macd', 'trend_macd_signal', 'trend_macd_diff', 'trend_sma_fast', 'trend_sma_slow', 'trend_ema_fast', 'trend_ema_slow

## Change point detection

In [4]:
# takes around 4 mins for me
df = df.sort_values("datetime")
df["log_return"] = np.log(df["close"]).diff().fillna(0)

print("change point start")
signal = df["log_return"].values.reshape(-1, 1)
# TODO: Pelt is taking too long to run for me, try running and see if its faster for u and also compare if it does give better results
#alg = rpt.Pelt(model="l2").fit(signal)
#change_points = alg.predict(pen=10)

alg = rpt.Binseg(model="l2").fit(signal)
change_points = alg.predict(n_bkps=30)
print("change point done")

segment_features = []
segment_idx = []
start = 0
for end in change_points:
    segment = df.iloc[start:end]
    features = {
        "mean_return": segment["log_return"].mean(),
        "volatility_log": segment["log_return"].std(),                
        "volatility_atr": segment["volatility_atr"].mean(), # range based vol
        "volatility_bbw": segment["volatility_bbw"].mean(), # band width
        "volatility_dcw": segment["volatility_dcw"].mean(), # high-low channel width
        "duration": len(segment),
    }
    segment_features.append(list(features.values()))
    segment_idx.append((start, end))
    start = end

print("begin kmeans")
# now that we found the main points of change for market trends, we can do clustering like KMeans to tag similar ones with same id
features_array = np.array(segment_features)
kmeans = KMeans(n_clusters=4, random_state=12) #since we predict theres 4 main market trends: bull, bear, high vol, low vol 
labels = kmeans.fit_predict(features_array)
print("completed kmeans")

df["trend_id"] = -1
for (start, end), label in zip(segment_idx, labels):
    df.iloc[start:end, df.columns.get_loc("trend_id")] = label

change point start
change point done
begin kmeans
completed kmeans


In [5]:
class StockTradingEnv(gym.Env):
    metadata = {"render_modes": ["human"]}

    def __init__(self, stock_data, transaction_cost_percent=0.005):
        super(StockTradingEnv, self).__init__()

        # Remove any empty DataFrames
        self.stock_data = {
            ticker: df for ticker, df in stock_data.items() if not df.empty
        }
        self.tickers = list(self.stock_data.keys())

        if not self.tickers:
            raise ValueError("All provided stock data is empty")

        # Calculate the size of one stock's data
        sample_df = next(iter(self.stock_data.values()))
        self.n_features = len(sample_df.columns)

        # Define action and observation space
        self.action_space = spaces.Box(
            low=-1, high=1, shape=(len(self.tickers),), dtype=np.float32
        )

        # Observation space: price data for each stock + balance + shares held + net worth + max net worth + current step
        self.obs_shape = self.n_features * len(self.tickers) + 2 + len(self.tickers) + 2
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, shape=(self.obs_shape,), dtype=np.float32
        )

        # Initialize account balance 1M$
        self.initial_balance = 1000000
        self.balance = self.initial_balance
        self.net_worth = self.initial_balance
        self.max_net_worth = self.initial_balance
        self.shares_held = {ticker: 0 for ticker in self.tickers}
        self.total_shares_sold = {ticker: 0 for ticker in self.tickers}
        self.total_sales_value = {ticker: 0 for ticker in self.tickers}

        # Set the current step
        self.current_step = 0

        # Calculate the minimum length of data across all stocks
        self.max_steps = max(0, min(len(df) for df in self.stock_data.values()) - 1)

        # Transaction cost
        self.transaction_cost_percent = transaction_cost_percent

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.balance = self.initial_balance
        self.net_worth = self.initial_balance
        self.max_net_worth = self.initial_balance
        self.shares_held = {ticker: 0 for ticker in self.tickers}
        self.total_shares_sold = {ticker: 0 for ticker in self.tickers}
        self.total_sales_value = {ticker: 0 for ticker in self.tickers}
        self.current_step = 0
        return self._next_observation(), {}

    def _next_observation(self):
        # initialize the frame
        frame = np.zeros(self.obs_shape)

        # Add stock data for each ticker
        idx = 0
        # Loop through each ticker
        for ticker in self.tickers:
            # Get the DataFrame for the current ticker
            df = self.stock_data[ticker]
            # If the current step is less than the length of the DataFrame, add the price data for the current step
            if self.current_step < len(df):
                frame[idx : idx + self.n_features] = df.iloc[self.current_step].values
            # Otherwise, add the last price data available
            elif len(df) > 0:
                frame[idx : idx + self.n_features] = df.iloc[-1].values
            # Move the index to the next ticker
            idx += self.n_features

        # Add balance, shares held, net worth, max net worth, and current step
        frame[-4 - len(self.tickers)] = self.balance  # Balance
        frame[-3 - len(self.tickers) : -3] = [
            self.shares_held[ticker] for ticker in self.tickers
        ]  # Shares held
        frame[-3] = self.net_worth  # Net worth
        frame[-2] = self.max_net_worth  # Max net worth
        frame[-1] = self.current_step  # Current step

        return frame

    def step(self, actions):
        # update the current step
        self.current_step += 1

        # check if we have reached the maximum number of steps
        if self.current_step > self.max_steps:
            return self._next_observation(), 0, True, False, {}

        current_prices = {}
        # Loop through each ticker and perform the action
        for i, ticker in enumerate(self.tickers):
            # Get the current price of the stock
            current_prices[ticker] = self.stock_data[ticker].iloc[self.current_step][
                "Close"
            ]
            # get the action for the current ticker
            action = actions[i]

            if action > 0:  # Buy
                # Calculate the number of shares to buy
                shares_to_buy = int(self.balance * action / current_prices[ticker])
                # Calculate the cost of the shares
                cost = shares_to_buy * current_prices[ticker]
                # Transaction cost
                transaction_cost = cost * self.transaction_cost_percent
                # Update the balance and shares held
                self.balance -= cost + transaction_cost
                # Update the total shares sold
                self.shares_held[ticker] += shares_to_buy

            elif action < 0:  # Sell
                # Calculate the number of shares to sell
                shares_to_sell = int(self.shares_held[ticker] * abs(action))
                # Calculate the sale value
                sale = shares_to_sell * current_prices[ticker]
                # Transaction cost, fixed fees...
                transaction_cost = sale * self.transaction_cost_percent
                # Update the balance and shares held
                self.balance += sale - transaction_cost
                # Update the total shares sold
                self.shares_held[ticker] -= shares_to_sell
                # Update the shares sold
                self.total_shares_sold[ticker] += shares_to_sell
                # Update the total sales value
                self.total_sales_value[ticker] += sale

        # Calculate the net worth
        self.net_worth = self.balance + sum(
            self.shares_held[ticker] * current_prices[ticker] for ticker in self.tickers
        )
        # Update the max net worth
        self.max_net_worth = max(self.net_worth, self.max_net_worth)
        # Calculate the reward
        reward = self.net_worth - self.initial_balance
        # Check if the episode is done
        done = self.net_worth <= 0 or self.current_step >= self.max_steps

        obs = self._next_observation()
        # for trend_ids
        trend_id = self.stock_data[ticker].iloc[self.current_step]["trend_id"]
        info = {"trend_id": trend_id}
        return obs, reward, done, False, info

    def render(self, mode="human"):
        # Print the current step, balance, shares held, net worth, and profit
        profit = self.net_worth - self.initial_balance
        print(f"Step: {self.current_step}")
        print(f"Balance: {self.balance:.2f}")
        for ticker in self.tickers:
            print(f"{ticker} Shares held: {self.shares_held[ticker]}")
        print(f"Net worth: {self.net_worth:.2f}")
        print(f"Profit: {profit:.2f}")

    def close(self):
        pass

In [6]:
def train(env_spawner, eval_env_spawner, config):
    print("Initializing...")

    # Unpack config
    log_dir = config["log_dir"]
    n_envs = config["n_envs"]
    ckpt = config.get("checkpoint", None)
    policy_args = config["policy_args"]
    algo_kwargs = config["algo_kwargs"]
    checkpoint_freq = config["checkpoint_freq"]
    eval_freq = config["eval_freq"]
    n_eval_episodes = config["n_eval_episodes"]
    n_train_timesteps = config["n_train_timesteps"]
    verbose_training = config["verbose_training"]

    # Setup
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)

    env = VecMonitor(DummyVecEnv(
        [env_spawner] * n_envs,
    ))
    eval_env = VecMonitor(DummyVecEnv([eval_env_spawner]))

    if ckpt is None:
        model = SAC(
            policy="MlpPolicy",
            env=env,
            policy_kwargs=policy_args,
            **algo_kwargs,
        )
    else:
        model = SAC.load(
            path=ckpt,
            env=env,
        )

    #if isinstance(model.replay_buffer, SERReplayBuffer):    
     #   model.replay_buffer.set_q_nets(model.q_net, model.q_net_target)

    checkpoint_callback = CheckpointCallback(
        save_freq=checkpoint_freq,
        save_path=log_dir,
        name_prefix=os.path.basename(log_dir),
        verbose=0,
    )

    eval_callback = EvalCallback(
        eval_env,
        best_model_save_path=log_dir,
        log_path=log_dir,
        eval_freq=eval_freq,
        n_eval_episodes=n_eval_episodes,
        deterministic=True,
        render=False,
        verbose=0,
    )

    # Train
    print("Training...")
    model.learn(
        total_timesteps=n_train_timesteps,
        callback=[
            eval_callback,
            checkpoint_callback,
        ],
        log_interval=1 if verbose_training else None,
        reset_num_timesteps=False,
        progress_bar=True if verbose_training else False,
    )

In [7]:
feature_cols = [
    "close",
    "volume_obv",
    "volatility_atr",
    "trend_macd",
    "trend_macd_signal",
    "trend_adx",
    "momentum_rsi",
    "momentum_ao",
    "trend_sma_fast",
    "trend_sma_slow",
    "trend_id"
]

# Make sure your DataFrame has no NaNs in these columns (drop or fill as needed)
df_selected = df[feature_cols].dropna().reset_index(drop=True)
df_selected = df_selected.rename(columns={"close": "Close"})

stock_data = { "SPY": df_selected }

def make_env():
    return StockTradingEnv(stock_data)

def make_eval_env():
    return StockTradingEnv(stock_data)  # could be different data if desired

In [8]:
base_config = {
    # TRAINING PARAMETERS
    "log_dir": "./train_logs/base",
    "n_envs": 4, # number of parallel environments to use for training
    "checkpoint": None, # path to a checkpoint to load from
    "checkpoint_freq": 10000, # save a model checkpoint every _ steps
    "eval_freq": 5000, # evaluate the model every _ steps
    "n_eval_episodes": 10, # number of episodes to evaluate the model on
    "n_train_timesteps": int(1e6), # total number of training steps
    "verbose_training": True,
    # RL PARAMETERS (all set to defaults right now, except for seed)
    "policy_args": {
        "net_arch": [64, 64],
        "activation_fn": nn.ReLU,
    },
    "algo_kwargs": {
        "learning_rate": 1e-4,
        "buffer_size": int(1e6),
        "learning_starts": 1000, # how many steps of the model to collect transitions for before learning starts
        "batch_size": 32,
        "tau": 1.0, # the soft update coefficient ("Polyak update", between 0 and 1) default 1 for hard update
        "gamma": 0.99,
        "train_freq": (4, 'step'), # Update the model every ``train_freq`` steps.
        "gradient_steps": 1, # How many gradient steps to do after each rollout
        "target_update_interval": int(1e4), # update the target network every ``target_update_interval`` environment steps.
        #"exploration_fraction": 0.1, # fraction of entire training period over which the exploration rate is reduced
        #"exploration_initial_eps": 1.0, # initial value of random action probability
        #"exploration_final_eps": 0.05, # final value of random action probability
        "seed": 42,
    }
}

In [ ]:
ser_config = base_config.copy()
ser_config["log_dir"] = "./train_logs/ser"
ser_config["algo_kwargs"]["replay_buffer_class"] = SERReplayBuffer

ser_config["priority_queue_size"] = 5000
ser_config["priority_queue_percent"] = 0.5


ser_config["algo_kwargs"]["replay_buffer_kwargs"] = {
    #"strategy": ser_config["strategy"],
    "priority_queue_size": ser_config["priority_queue_size"],
    "priority_queue_percent" : ser_config["priority_queue_percent"]
}

train(make_env, make_eval_env, ser_config) 

Initializing...


Output()

set_curr_trend

in set_curr_trend

start rescore

heapify

heapify done

Training...


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


KeyboardInterrupt: 

In [ ]:
def plot_evaluations(log_dir):
    evaluations = np.load(os.path.join(log_dir, "evaluations.npz"))
    timesteps = evaluations["timesteps"]
    results_all = evaluations["results"]  # shape: (eval_rounds, episodes_per_eval)
    
    mean_returns = np.mean(results_all, axis=1)
    std_returns = np.std(results_all, axis=1)
    num_episodes_averaged = results_all.shape[1]
    print("Evaluations: min: {}, max: {}, std: {}".format(
        np.min(results_all), np.max(results_all), np.std(results_all)))
        
    plt.figure(figsize=(5, 5))
    plt.plot(timesteps, mean_returns, label='Mean Return')
    plt.fill_between(timesteps, mean_returns - std_returns, mean_returns + std_returns,
                     alpha=0.3, label='Std Dev')
    plt.xlabel('Timesteps')
    plt.ylabel('Returns (averaged over {} episodes)'.format(num_episodes_averaged))
    plt.title('Evaluation Returns')
    plt.legend()
    plt.grid()

    plt.savefig(os.path.join(log_dir, "evaluations.png"))
    plt.show()

In [ ]:
plot_evaluations(ser_config["log_dir"])